In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import emoji

In [2]:
df = pd.read_excel('labeled data.xlsx')

In [3]:
print(f'Строк: {df.shape[0]:,}, столбцов: {df.shape[1]}')

Строк: 2,373, столбцов: 8


In [4]:
df.head()

,id,rating,comment,Нравится скорость отработки заявок,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Понравилось выполнение заявки,Вопрос решен
0,2945792,5,спасибо,NaN,NaN,NaN,1.0,NaN
1,3234340,5,спасибо!,NaN,NaN,NaN,1.0,NaN
2,3380332,5,Отлично,NaN,NaN,NaN,1.0,NaN
3,3381812,5,Благодарю за оперативное решение проблемы !,1.0,NaN,NaN,NaN,NaN
4,3461991,5,Прекрасный специалист! Побольше таких,NaN,NaN,1.0,NaN,NaN


In [ ]:
duplicates_comments = df[df['comment'].duplicated(keep=False)]['comment']

mask_short = df['comment'].str.len() < 10
short_comments = df.loc[mask_short, ['comment']].copy()
short_comments['length'] = short_comments['comment'].str.len()

duplicates_count = duplicates_comments.shape[0]

print(f"Всего дубликатов отзывов: {duplicates_count}")
print("Примеры дубликатов:")
print(duplicates_comments.head(10).to_string(index=False))

Всего дубликатов отзывов: 702
Примеры дубликатов:
                                     спасибо
                                    спасибо!
                                    Отлично 
Благодарю за оперативное решение проблемы ! 
       Прекрасный специалист! Побольше таких
                                    Спасибо 
                     пересчет, и скорость \n
         Спасибо за обновление покраски стен
                            выполнили быстро
                                   Спасибо! 


In [6]:
df_no_duplicates = df[~df['comment'].duplicated(keep=False)].copy()

In [ ]:
output_path = 'data_new.xlsx'
df_no_duplicates.to_excel(output_path, index=False)

### Переразметим в ручную данные 

In [9]:
df = pd.read_excel('data_new_final.xlsx')

In [10]:
df.head()

,id,rating,comment,Нравится скорость отработки заявок,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Понравилось выполнение заявки,Вопрос решен
0,3872124,5,"Большое спасибо Александру,за оперативность и ...",1.0,1.0,1.0,NaN,1.0
1,3872129,5,благодарю,NaN,NaN,NaN,1.0,1.0
2,3873692,5,оперативно и качественно. спасибо,1.0,1.0,NaN,NaN,1.0
3,3875262,5,"благодарю, всего доброго!",NaN,NaN,NaN,1.0,1.0
4,3877334,3,К мастеру вопросов нет.,NaN,NaN,1.0,NaN,1.0


In [11]:
print(f'Строк: {df.shape[0]:,}, столбцов: {df.shape[1]}')

Строк: 1,669, столбцов: 8


In [12]:
def replace_emoji_to_text(text):
    return ''.join(
        emoji.demojize(char, delimiters=(" :", ": ")) if char in emoji.EMOJI_DATA else char
        for char in text
    )
df['comment'] = df['comment'].apply(replace_emoji_to_text)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.utils import resample
from skmultilearn.model_selection import iterative_train_test_split

np.random.seed(42)
df = (
    df.sample(frac=1, random_state=42)
      .reset_index(drop=True)
)

label_cols = [c for c in df.columns if c not in ["id", "rating", "comment"]]
X = df["comment"].astype(str).values
Y = df[label_cols].fillna(0).astype(int).values

X_tr, y_tr, X_te, y_te = iterative_train_test_split(X.reshape(-1,1), Y, test_size = 0.2)
                                                    
                                                    
X_tr = X_tr.ravel()
X_te = X_te.ravel()

train_df = pd.DataFrame({"comment": X_tr})
train_df[label_cols] = y_tr
test_df  = pd.DataFrame({"comment": X_te})
test_df[label_cols]  = y_te

train_df.to_excel("train_80.xlsx", index=False)
test_df.to_excel("test_20.xlsx", index=False)

print("Train shape:", train_df.shape)
print(train_df[label_cols].sum())
print("Test shape:", test_df.shape)
print(test_df[label_cols].sum())

/Users/snow0w/repos/ADD/practiceSummerSamolet/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train shape: (1335, 6)
Нравится скорость отработки заявок       548
Нравится качество выполнения заявки      178
Нравится качество работы сотрудников     313
Понравилось выполнение заявки            246
Вопрос решен                            1162
dtype: int64
Test shape: (334, 6)
Нравится скорость отработки заявок      137
Нравится качество выполнения заявки      44
Нравится качество работы сотрудников     78
Понравилось выполнение заявки            61
Вопрос решен                            290
dtype: int64


In [ ]:
print("\nКодируем LaBSE-эмбеддинги …")
labse  = SentenceTransformer("sentence-transformers/LaBSE")
emb_tr = labse.encode(train_df["comment"].tolist(), show_progress_bar=True)
emb_te = labse.encode(test_df["comment"].tolist(),  show_progress_bar=True)

print("Обучаем KNN-5 (One-vs-Rest) …")
clf    = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
clf.fit(emb_tr, train_df[label_cols].values)

y_pred   = clf.predict(emb_te)
macro_f1 = f1_score(test_df[label_cols].values, y_pred, average="macro")
print(f"\nBaseline macro-F1 = {macro_f1:.3f}")


Кодируем LaBSE-эмбеддинги …


Batches: 100%|██████████| 11/11 [00:00<00:00, 18.40it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Обучаем KNN-5 (One-vs-Rest) …

Baseline macro-F1 = 0.795


# GIGA chat generation

In [43]:
import requests
from pathlib import Path
from gigachat import GigaChat

url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

home = Path.home()
file_path = home / '.gigachatAuthKey'
key = ''
try:
    with open(file_path, 'r') as f:
        key = f.read().strip()
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

payload={
  'scope': 'GIGACHAT_API_PERS'
}
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'RqUID': 'aa1bcbba-9fc7-4381-b82b-1d54308c86a8',
  'Authorization': f'Basic {key}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

acc_key = response.json()['access_token']

/Users/snow0w/repos/ADD/practiceSummerSamolet/venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
import re, time, random
import pandas as pd
from tqdm import tqdm
from openai import OpenAI, APIError, APIConnectionError, RateLimitError

client = GigaChat(
   credentials=key,
   verify_ssl_certs=False,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-2",
    )
train_df = pd.read_excel("train_80.xlsx")

SYSTEM = ("Ты пишешь короткие отзывы жильцов "
          "управляющей компании. Пиши строго на русском.")

PROMPT_BASE = (
    "Пример отзыва:\n«{ex}»\n\n"
    "{emoji_hint}"
    "Напиши 1 уникальный отзыв по смыслу, похожий на пример (1-6 предложений). "
    "Не копируй пример. Можно перефразировать, добавить детали, но отзыв должен быть уникальным."
)

EMOJI_RE = re.compile(r":[a-z0-9_]+:", flags=re.I)

def gpt_review(example):
    has_code = bool(EMOJI_RE.search(example))
    emoji_hint = ("Если в примере есть код эмодзи вида :smile:, "
                  "то сохрани стиль — вставь похожий код эмодзи.\n\n"
                  ) if has_code else ""
    prompt = PROMPT_BASE.format(ex=example, emoji_hint=emoji_hint)
    print(SYSTEM + " " +prompt)

    r = client.chat(
        prompt
    )
    return r.choices[0].message.content

In [ ]:
synthetic_comments = []
for comment in tqdm(train_df['comment'], desc="Генерируем синтетику"):
    new_comment = gpt_review(comment)
    if new_comment:
        synthetic_comments.append(new_comment)
    else:
        synthetic_comments.append("")

synthetic_df = train_df.copy()
synthetic_df['comment'] = synthetic_comments

full_df = pd.concat([train_df, synthetic_df], ignore_index=True)

full_df.to_excel("train_plus_synthetic.xlsx", index=False)

Генерируем синтетику:   0%|          | 0/1335 [00:00<?, ?it/s]

Ты пишешь короткие отзывы жильцов управляющей компании. Пиши строго на русском. Пример отзыва:
«очень оперативно решили проблему и восстановили освещение в холле.»

Напиши 1 уникальный отзыв по смыслу, похожий на пример (1-2 предложений). Не копируй пример. Можно перефразировать, добавить детали, но отзыв должен быть уникальным.


Генерируем синтетику:   0%|          | 1/1335 [00:00<08:21,  2.66it/s]

Ты пишешь короткие отзывы жильцов управляющей компании. Пиши строго на русском. Пример отзыва:
«днём звонили был занят за рулём извините,,,, вечером пришли домой всё пересмотрели проверили вашу информацию спасибо нам всё понятно всё на каждый вопрос Вы ответили!!! спасибо Вам большое.!!!»

Напиши 1 уникальный отзыв по смыслу, похожий на пример (1-2 предложений). Не копируй пример. Можно перефразировать, добавить детали, но отзыв должен быть уникальным.


Генерируем синтетику:   0%|          | 3/1335 [00:00<06:18,  3.52it/s]

...

Генерируем синтетику: 100%|█████████▉| 1333/1335 [05:39<00:00,  4.36it/s]

Ты пишешь короткие отзывы жильцов управляющей компании. Пиши строго на русском. Пример отзыва:
«спасибо за оперативность)»

Напиши 1 уникальный отзыв по смыслу, похожий на пример (1-2 предложений). Не копируй пример. Можно перефразировать, добавить детали, но отзыв должен быть уникальным.
Ты пишешь короткие отзывы жильцов управляющей компании. Пиши строго на русском. Пример отзыва:
«удивлен, что так быстро среагировали»

Напиши 1 уникальный отзыв по смыслу, похожий на пример (1-2 предложений). Не копируй пример. Можно перефразировать, добавить детали, но отзыв должен быть уникальным.


Генерируем синтетику: 100%|█████████▉| 1334/1335 [05:39<00:00,  4.50it/s]

Ты пишешь короткие отзывы жильцов управляющей компании. Пиши строго на русском. Пример отзыва:
«очень хороший сотрудник. помогла установить приложение ключ, объяснили как работает приложение, сделали три ключа. доброжелательное отношение, всё оперативно.»

Напиши 1 уникальный отзыв по смыслу, похожий на пример (1-2 предложений). Не копируй пример. Можно перефразировать, добавить детали, но отзыв должен быть уникальным.


Генерируем синтетику: 100%|██████████| 1335/1335 [05:40<00:00,  3.93it/s]


In [ ]:
print("\nКодируем LaBSE-эмбеддинги …")
# labse  = SentenceTransformer("sentence-transformers/LaBSE")
emb_tr = labse.encode(full_df["comment"].tolist(), show_progress_bar=True)
emb_te = labse.encode(test_df["comment"].tolist(),  show_progress_bar=True)

print("Обучаем KNN-5 (One-vs-Rest) …")
clf    = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
clf.fit(emb_tr, full_df[label_cols].values)

y_pred   = clf.predict(emb_te)
macro_f1 = f1_score(test_df[label_cols].values, y_pred, average="macro")
print(f"\nBaseline macro-F1 = {macro_f1:.3f}")


Кодируем LaBSE-эмбеддинги …


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

Batches: 100%|██████████| 11/11 [00:00<00:00, 24.31it/s]


Обучаем KNN-5 (One-vs-Rest) …

Baseline macro-F1 = 0.788
